In [1]:
from xml.dom import minidom


In [2]:
xmldoc = minidom.parse("../disprot/disprot_v6.02.xml")

In [23]:
prots = []

In [24]:
def get_val(parent, tagname):
    return parent.getElementsByTagName(tagname)[0].firstChild.nodeValue

In [29]:
for xmlprot in xmldoc.getElementsByTagName("protein"):
    xmlgeneral = xmlprot.getElementsByTagName("general")[0]
    prot = {}
    prot["name"] = get_val(xmlgeneral, "name")
    prot["regions"] = []
    for xmlregion in xmlprot.getElementsByTagName("regions")[0].getElementsByTagName("region"):
        region = {}
        region["type"] = get_val(xmlregion, "type")
        prot["regions"].append(region)
    prots.append(prot)


In [30]:
prots

[{'name': '60S acidic ribosomal protein P1-B',
  'regions': [{'type': 'Disordered'}]},
 {'name': '60S acidic ribosomal protein P2-beta',
  'regions': [{'type': 'Disordered - Molten Globule'}]},
 {'name': 'Early E2A DNA-binding protein',
  'regions': [{'type': 'Disordered - Extended'},
   {'type': 'Ordered'},
   {'type': 'Ordered'},
   {'type': 'Ordered'},
   {'type': 'Disordered - Extended'},
   {'type': 'Disordered - Extended'},
   {'type': 'Ordered'}]},
 {'name': 'Cathelicidin antimicrobial peptide', 'regions': []},
 {'name': 'Antibacterial protein LL-37',
  'regions': [{'type': 'Disordered - Extended'}]},
 {'name': 'Antitermination protein N', 'regions': [{'type': 'Disordered'}]},
 {'name': 'Cytochrome c', 'regions': [{'type': 'Disordered'}]},
 {'name': 'DNA-(apurinic or apyrimidinic site) lyase',
  'regions': [{'type': 'Disordered - Extended'},
   {'type': 'Disordered - Extended'},
   {'type': 'Disordered'},
   {'type': 'Disordered - Extended'},
   {'type': 'Disordered - Extended'}